# Gezichtsherkenning


## Loading packages
Use conda environment: *xtopia-video*

In [ ]:
import cv2
from deepface import DeepFace
import matplotlib.pyplot as plt

# Van statische image deepface analyse maken

In [ ]:
imgpath = 'Images/Neutral Face.jpg'  #put the image where this file is located and put its name here
image = cv2.imread(imgpath)

analyze = DeepFace.analyze(image, actions = ['emotion'])  # analyseren van de image met de analyze functie van de deepface package

print(analyze)

In [ ]:
print(f"dominante emotie: {analyze['dominant_emotion']}") # printen dominante emotie

In [ ]:
# font = cv2.FONT_HERSHEY_SIMPLEX
# cv2.putText(image,
#             analyze['dominant_emotion'],
#             (0, 50),
#             font, 1,
#             (0,0,255),
#             2,
#             cv2.LINE_4);
#
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_YCrCb2RGB))

# Deepface analyse van Webcam input

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX # font instellen voor emotie op webcam beeld te printen
delay = 0   # timer om de deepface analyse maar op de zoveel keren van de while loop uit te laten voeren
            # bv door maar op de 50 keer door de while te gaan een analyse maken, om geheugen te besparen


face_cascade_name = cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml'  #getting a haarcascade xml file
face_cascade = cv2.CascadeClassifier()  #processing it for our project
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):  #adding a fallback event
    print("Error loading xml file")

video = cv2.VideoCapture(0) # parameter 0 omdat we maar 1 camera hebben (1 webcam), indien bv. 2 camera's -> parameter 1, enz ...

# Check if the webcam is opened correctly
if not video.isOpened():
    raise IOError("Cannot open webcam")

while video.isOpened():  #checking if are getting video feed and using it
    ret,frame = video.read() #  ret is a Boolean value returned by the read function, and it indicates whether or not the frame was captured                                successfully. If the frame is captured correctly, it's stored in the variable frame.

    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  #changing the video to grayscale to make the face analisis work properly
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for x,y,w,h in faces:

      cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 1)  # vierkant rond gezicht zetten + kleur

    if delay % 10 == 0:    # voer analyse maar uit op de 50 keer door de while te gaan
        #making a try and except condition in case of any errors
        try:
          analyze = DeepFace.analyze(frame, actions = ['emotion'])
          result = analyze['dominant_emotion']
        except:
          result = "no face"
          #print("no face")

    # emotie op webcam beeld afdrukken
    cv2.putText(frame,
                result,
                (0, 50),
                font, 1,
                (0,0,255),
                2,
                cv2.LINE_4)

    if(delay < 500):
        delay+=1 # timer incrementeren
    else:
        delay = 0 # timer resetten, zodat deze niet te groot wordt

    #this is the part where we display the output to the user
    cv2.imshow('video', frame)

    if cv2.waitKey(1) == ord('q'): # klik op 'q' toets op af te sluiten
        break

video.release()
cv2.destroyAllWindows()

# In klassen (cleaned)

In [ ]:
global delay # timer om de deepface analyse maar op de zoveel keren van de while loop uit te laten voeren
            # bv door maar op de 50 keer door de while te gaan een analyse maken, om geheugen te besparen
delay = 0

face_cascade_name = cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml'  #getting a haarcascade xml file
face_cascade = cv2.CascadeClassifier()  #processing it for our project


def emotion_detection():

    if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):  #adding a fallback event
        print("Error loading xml file")

    video = cv2.VideoCapture(0)  # parameter 0 omdat we maar 1 camera hebben (1 webcam), indien bv. 2 camera's -> parameter 1, enz ...

    # Check if the webcam is opened correctly
    if not video.isOpened():
        raise IOError("Cannot open webcam")

    while video.isOpened():  #checking if are getting video feed and using it
        isFrameCaptured, frame = video.read()  #  ret is a Boolean value returned by the read function, and it indicates whether or not the frame was captured successfully. If the frame is captured correctly, it's stored in the variable frame.

        if not isFrameCaptured:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        detect_face(frame)
        check_emotion_every_X_seconds(frame, delay)

        #this is the part where we display the output to the user
        cv2.imshow('video', frame)

        if cv2.waitKey(1) == ord('q'):  # klik op 'q' toets op af te sluiten
            break

    video.release()
    cv2.destroyAllWindows()


def check_emotion(frame):
    #making a try and except condition in case of any errors
    try:
        analyze = DeepFace.analyze(frame, actions=['emotion'])
        result = analyze['dominant_emotion']
    except:
        result = "no face"

    return result

def print_emotion_on_screen(frame, result):
    # emotie op webcam beeld afdrukken
    cv2.putText(frame,
                result,
                (0, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, # font instellen (Hershey Simplex)
                (0, 0, 255),
                2,
                cv2.LINE_4)

def detect_face(frame):
    
    gray = cv2.cvtColor(frame,
                        cv2.COLOR_BGR2GRAY)  #changing the video to grayscale to make the face analisis work properly
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    for x, y, w, h in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 1)  # vierkant rond gezicht zetten + kleur

def check_emotion_every_X_seconds(frame, timer):
    if timer % 10 == 0:  # voer analyse maar uit op de 10 keer (1 sec) door de while te gaan
        result = check_emotion(frame)
    print_emotion_on_screen(frame, result)
    if (timer < 500):
        timer += 1  # timer incrementeren
    else:
        timer = 0  # timer resetten, zodat deze niet te groot wordt

    return result



In [ ]:
emotion_detection()